In [1]:
!pip install transformers
!pip install pycaret

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 4.2 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of category-encoders to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.1/486.1 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.8/21.8 MB 98.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 kB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 125.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import transformers
from transformers import AutoModel,BertTokenizerFast
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
import warnings
warnings.filterwarnings('ignore')
import torch
import torch.nn as nn
#specify gpu
device=torch.device('cuda')

SyntaxError: invalid syntax (<ipython-input-2-8922c6d527f4>, line 14)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/projects


/content/drive/MyDrive/Colab Notebooks/projects


In [ ]:
true_data=pd.read_csv('a1_True.csv')
fake_data=pd.read_csv('a2_Fake.csv')

true_data['Target']=['True']*len(true_data)
fake_data['Target']=['Fake']*len(fake_data)
#we concate the 2 datasets and shuffle them into single datset data
data= pd.concat([true_data,fake_data]).sample(frac=1).reset_index(drop=True)
print(data.shape)
data.head()

(44898, 5)


,title,text,subject,date,Target
0,"Trump was being a gentleman, UK's May says on ...",LONDON (Reuters) - Donald Trump was being a ge...,politicsNews,"March 21, 2017",True
1,ICAN leader 'delighted' with Nobel Peace Prize...,OSLO (Reuters) - The leader of the Internation...,worldnews,"October 6, 2017",True
2,CROCODILE TEARS: Watch Obama Use Phony Outrage...,"Following Obama s phony tears, he launched dir...",left-news,"Jan 5, 2016",Fake
3,Illinois governor announces veto of $5 billion...,CHICAGO (Reuters) - Illinois Governor Bruce Ra...,politicsNews,"July 4, 2017",True
4,ACTRESS JODIE FOSTER Weighs In On The Phony ‘W...,Jodie Foster gives a dose of common sense to ...,left-news,"Apr 23, 2016",Fake


In [ ]:
# prompt: convert the true or false into 0,1

data['label']=pd.get_dummies(data['Target'])['Fake']
data.head()

,title,text,subject,date,Target,label
0,"Trump was being a gentleman, UK's May says on ...",LONDON (Reuters) - Donald Trump was being a ge...,politicsNews,"March 21, 2017",True,False
1,ICAN leader 'delighted' with Nobel Peace Prize...,OSLO (Reuters) - The leader of the Internation...,worldnews,"October 6, 2017",True,False
2,CROCODILE TEARS: Watch Obama Use Phony Outrage...,"Following Obama s phony tears, he launched dir...",left-news,"Jan 5, 2016",Fake,True
3,Illinois governor announces veto of $5 billion...,CHICAGO (Reuters) - Illinois Governor Bruce Ra...,politicsNews,"July 4, 2017",True,False
4,ACTRESS JODIE FOSTER Weighs In On The Phony ‘W...,Jodie Foster gives a dose of common sense to ...,left-news,"Apr 23, 2016",Fake,True


In [ ]:
data["label"]=data["label"].astype(int)

In [ ]:
data.head()

,title,text,subject,date,Target,label
0,"Trump was being a gentleman, UK's May says on ...",LONDON (Reuters) - Donald Trump was being a ge...,politicsNews,"March 21, 2017",True,0
1,ICAN leader 'delighted' with Nobel Peace Prize...,OSLO (Reuters) - The leader of the Internation...,worldnews,"October 6, 2017",True,0
2,CROCODILE TEARS: Watch Obama Use Phony Outrage...,"Following Obama s phony tears, he launched dir...",left-news,"Jan 5, 2016",Fake,1
3,Illinois governor announces veto of $5 billion...,CHICAGO (Reuters) - Illinois Governor Bruce Ra...,politicsNews,"July 4, 2017",True,0
4,ACTRESS JODIE FOSTER Weighs In On The Phony ‘W...,Jodie Foster gives a dose of common sense to ...,left-news,"Apr 23, 2016",Fake,1


In [ ]:
data["Target"].value_counts()

,count
Target,
Fake,23481
True,21417


In [ ]:
# Train-Validation-Test set split into 70:15:15 ratio
#Train-Temp split
#we are splitting training and temp in 70:30
train_text,temp_text,train_labels,temp_labels =train_test_split(data['text'],data['label'],random_state=2018,test_size=0.3,stratify=data['label'])
#we are spliting temp in validation and test
#validation test split
val_text,test_text,val_labels,test_labels =train_test_split(temp_text,temp_labels,random_state=2018,test_size=0.5,stratify=temp_labels)

In [ ]:
#Loading pretrained bert model

In [ ]:
bert=AutoModel.from_pretrained('bert-base-cased')
tokenizer=BertTokenizerFast.from_pretrained('bert-base-cased')